# Graph Magma Class 

In [9]:
class graphMagma:
    from sage.matrix.constructor import block_matrix, zero_matrix, identity_matrix, matrix
    from sage.rings.integer_ring import ZZ
    #Create a global variable to store the generators
    generators = []
    Inc = matrix(ZZ, 0, 0, [])
    multiplication_table = matrix(SR,0,0,[])
    #Create a global variable to store the graph
    graph = []
    def __init__(self, graph:list):
        self.graph = graph
        self.generators = self.getGenerators()
        self.Inc = self.getIncidenceMatrix()
        self.multiplication_table = self.getMultiplicationTable()
    def getGenerators(self):
        self.generators = []
        for i in range(len(self.graph)):
            for j in range(self.graph[i][1]):
                self.generators.append(var(f"a_{i}_{j}", latex_name=fr"\alpha_{{{i},{j}}}"))
        return self.generators
    def getGraph(self):
        return self.graph
    def incidenceMatrix(self):
        G=self.graph
        I=self.Inc
        for g in G:
            if g[0] == 'K':
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1], g[1], [1]*(g[1]*g[1]))]])
            else:
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1],g[1])]])  
        self.Inc = I
    def getIncidenceMatrix(self):
        self.incidenceMatrix()
        return self.Inc  
    def multiplicationTable(self):
        I=self.Inc
        M=matrix(SR,I.nrows(),I.nrows())
        self.getGenerators()
        for i in range(I.nrows()):
            for j in range(I.nrows()):
                if I[i,j]==0:
                    M[i,j]=self.generators[j]
                else:
                    M[i,j]=self.generators[i]
        self.multiplication_table = M
    def getMultiplicationTable(self):
        self.multiplicationTable()
        return self.multiplication_table

# Graph Magma Algebra Class

In [41]:
from sage.algebras.free_algebra import FreeAlgebra
from sage.algebras.free_algebra_quotient import FreeAlgebraQuotient
from sage.structure.element import Element
from random import choice
relations=[]
class GraphMagmaAlg:
    def __init__(self,base_ring,GM:graphMagma):
        R=FreeAlgebraQuotient
        self.base_ring=base_ring
        self.generators=GM.getGenerators()
        self.generators_str=[str(g) for g in self.generators]
        self.graph=GM.getGraph()
        self.multiplication_table=GM.getMultiplicationTable()
        # Determine the number of generators from the list
        n = len(self.generators)
        # Create a string for the generator names
        generator_names_str =', '.join(self.generators_str)
        #g_str=f"R.<{generator_names_str}>=FreeAlgebraQuotient(A,self.mons,self.mats)";print(g_str)
        # Create the FreeAlgebra instance
        A = FreeAlgebra(base_ring, n,generator_names_str)
        self.alg=A
        F=A.monoid() #generator_names_str=F.gens();print(F.gens())
        self.mons=[F(1)]+list(F.gens())
        self.mats=self.build_relations()
        self.magma_algebra=FreeAlgebraQuotient(A, self.mons, self.mats, names=self.generators_str)
        splt=[g[1] for g in self.graph]
        L=list(self.magma_algebra.gens()).copy()
        self.magma_algebra_gens=self.split_list(L,list(splt))
        self.local_decomposition=self.getDecomposition()
        self.left_indecomposables=self.getLeftIndecomposables()
        
    
    def __contains__(self, item):
        return item in self.magma_algebra
    
    def __str__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"
    def __repr__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"

    def build_relations(self):
        act=[];L=[];idx=0
        n=len(self.generators)
        self.relations=[];M=MatrixSpace(self.base_ring,n+1);I=identity_matrix(n+1)
        T=self.multiplication_table
        for j in range(T.nrows()):
            L=list(I[j+1])
            for i in range(T.ncols()):
                idx=self.generators.index(T[i,j])
                L+=list(I[idx+1])
            act.append(M(L))
        return act
            
    def split_list(self,L, lengths):
        result = []
        index = 0
        for length in lengths:
            sublist = L[index:index + length]
            result.append(sublist)
            index += length
        return result

    def getDecomposition(self):
        R=self.magma_algebra;g=self.magma_algebra_gens
        L=[R(f[0]) for f in g]
        ideal_generators=[L[0]]+[L[i]-L[i-1] for i in [1..len(L)-1]]
        return ideal_generators

    def getLeftIndecomposables(self):
        R=self.magma_algebra
        IG=self.local_decomposition
        return [R.lideal(g) for g in IG]


In [42]:
M=graphMagma([('K',2),('N',3)])
GM=GraphMagmaAlg(QQ,M);GM.left_indecomposables

AttributeError: 'FreeAlgebraQuotient_with_category' object has no attribute 'lideal'

In [4]:
R=GM.magma_algebra
R

Free algebra quotient on 5 generators ('a_0_0', 'a_0_1', 'a_1_0', 'a_1_1', 'a_1_2') and dimension 6 over Rational Field

In [6]:
g=GM.magma_algebra_gens
(g[0][0]*g[1][0])

a_0_0

In [45]:
g[0][0]^2==g[0][0]

True

In [11]:
g

[[a_0_0, a_0_1], [a_1_0, a_1_1, a_1_2]]

In [13]:
(1-g[0][0])**2==1-g[0][0]

True